In [16]:
GUI()

# Model Simulation

Here you run the simulation. Either choose a scenario or build your own.

In [17]:
import ipywidgets as widgets
from IPython.display import display, Markdown
import time

def GUI(): 
    #-----------Choose scenario tab----------------------------------
    runbtn = widgets.Button(
            description='Run Simulation',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='', # (FontAwesome names without the `fa-` prefix)
            layout = {'font-size':'24px'}
            )

    scenario_selection = widgets.ToggleButtons(
             options=['1', '2', '3', '4'],
             description='',
             disabled=False,
             button_style='', # 'success', 'info', 'warning', 'danger' or ''
             tooltips=['Description of scenario 1', 'Description of scenario 2', 'Description of scenario 3', 'Description of scenario 4'],
             #icons=['check'] * 4
             layout={'width': 'max-content',
                    'margin':'15px',
                    'margin-bottom': '30px'},)

    chooselayout = widgets.Layout(display='flex',
                        flex_flow='column',
                        align_items='center',
                        border='none',
                        width='100%',
                        )

    scenariobox = widgets.VBox(
        [widgets.HTML(
        value="<h2> Choose a scenario </h2>",
        ),
         scenario_selection, 
         runbtn,],
        layout = chooselayout
        )

    #------------------ Build scenario tab ----------------------------------

    region_dropdown = widgets.Dropdown(options=['Stockholm'],
                                       description='Region:',
                                       disabled=False,
                                      )

    housenew = widgets.IntText(value=0,layout={'width': '70px'})
    housenewbox = widgets.Box([widgets.Label('New houses (electrical heating):'),
                            housenew],)

    houseold = widgets.IntText(value=0,layout={'width': '70px'})
    houseoldbox = widgets.Box([widgets.Label('Old houses (electrical heating):'),
                            houseold],)

    housemix = widgets.IntText(value=0,layout={'width': '70px'})
    housemixbox = widgets.Box([widgets.Label('Mixed age houses (district heating):'),
                            housemix],)

    apart = widgets.IntText(value=0,layout={'width': '70px'})
    apartbox = widgets.Box([widgets.Label('New apartments (district heating):'),
                          apart])

    office = widgets.IntText(value=0,layout={'width': '70px'})
    apartbox = widgets.Box([widgets.Label('New apartments (district heating):'),
                          apart])

    resloadbox = widgets.VBox([housenewbox,
                               houseoldbox,
                               housemixbox,
                               apartbox],
                              layout = {'margin':'15px'}
                              )
    offices = widgets.IntText(value = 0,layout={'width': '70px'})
    officebox = widgets.HBox([widgets.Label('Number of offices:'),
                             offices],
                             layout = {'margin':'15px'}
                            )

    PV69 = widgets.IntText(value=0,
                             min=0,
                             description='69 m2:',
                             disabled=False,
                             layout={'width': 'max-content'}
                                                      )
    PV400 = widgets.IntText(value=0,
                             min=0,
                             description='400 m2:',
                             disabled=False,
                             layout={'width': 'max-content'}
                                                      )
    PV868 = widgets.IntText(value=0,
                             min=0,
                             description='868 m2:',
                             disabled=False,
                             layout={'width': 'max-content'}
                                                      )
    PVbox = widgets.HBox([widgets.Label('Number of solar cells:'),
                         widgets.HBox([PV69,
                                       PV400,
                                       PV868,
                         ])],
                        layout = {'margin':'15px'})

    EVnumber = widgets.IntText(value = 0, 
                               #description = 'Number of EVs in substation:',
                              layout={'width': '70px'})
    EVparking = widgets.IntText(value = 0, 
                               #description = 'Number of Parking spaces in substation:',
                               layout={'width': '70px'})
    EVkwhkm = widgets.FloatText(value = 0.2, 
                               #description = 'Number of Parking spaces in substation:',
                               layout={'width': '70px'})

    EVbox = widgets.Box([widgets.Label('Number of EVs:'),EVnumber,
                         widgets.Label('Number of parking spaces:'),EVparking,
                         widgets.Label('Mean consumption of EVs (kWh/km):'),EVkwhkm],
                       layout = {'margin':'15px'})

    flexcheck = widgets.Checkbox(value=False,
                                 description='Demand Side Flexibility',
                                 disabled=False,
                                 indent=False
                                )

    flexpercent = widgets.IntText(value=30,
                             min=0,
                             disabled= True,
                            layout={'width': '70px'}
                            )

    flexdays = widgets.IntText(value=17,
                             min=0,
                             disabled= True,
                            layout={'width': '70px'}
                            )
    flexloadpercent = widgets.IntText(value=10,
                             min=0,
                             disabled= True,
                            layout={'width': '70px'}
                            )

    flexbox = widgets.VBox([flexcheck,
                         widgets.HBox([widgets.Label('Flex extent (%)'),flexpercent, 
                         widgets.Label('Days to be flexible/year:'),flexdays, 
                         widgets.Label('Percentage of loads that are flexible:'),flexloadpercent,])
                        ], 
                        layout = {'margin':'15px'})

    EEcheck = widgets.Checkbox(value=False,
                               description='Energy efficiency',
                               disabled=False,
                               indent=False,
                               layout={'font-size': '16px'},
                                )

    EEreduction = widgets.IntSlider(min = 0,
                                    max = 100,
                                    value = 30,
                                    disabled = True, 
                                   )

    EEloads = widgets.IntSlider(min = 0,
                                max = 0,
                                disabled = True,
                               )

    EEbox = widgets.VBox([EEcheck, 
                         widgets.VBox([widgets.HBox([widgets.Label('Number of loads to reduce'),EEloads]),
                                       widgets.HBox([widgets.Label('Reduction of consumption(%)'),EEreduction]),
                                      ],
                        layout={'width': 'max-content'},
                         ), 
                         ],
                       layout = {'margin':'15px'})

    runbtn2 = widgets.Button(
            description='Run Simulation',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='', # (FontAwesome names without the `fa-` prefix)
            layout = {'font-size':'24px'}
            )


    btnbox = widgets.HBox([runbtn2],
                        layout = {'width': 'max-content',
                                  'align-items':'stretch',
                                  'margin':'15px'})

    custom_layout = widgets.Layout(display='flex',
                                   flex_flow='column',
                                   align_items='stretch',
                                   border='none',
                                   width='99%',
                        )

    create_scenario = widgets.Box([region_dropdown, 
                                    resloadbox, 
                                    officebox, 
                                    PVbox, 
                                    EVbox, 
                                    EEbox, 
                                    flexbox, 
                                    btnbox
                                   ],
                                  layout = custom_layout)



    #-------------------Events and Output -----------------------------------
    max_count = 500
    f = widgets.IntProgress(min=0, 
                            max=max_count,
                            description = 'Loading:'
                           ) # instantiate the bar

    def progress(obj):
        display(f) # display the bar
        count = 0
        while count <= max_count:
            f.value += 1 # signal to increment the progress bar
            time.sleep(0.1)
            count += 1

    def run_model(event, scenario = None):
        if scenario:
            print('Running scenario ', scenario)
            region = 'Stockholm'
            # Loads
            office = 1
            load_dict = {'HouseNew': 2,
                         'HouseOld' : 2, 
                         'HouseDH': 2, 
                         'ApartmentNewDH': 2
                        }
            # PV
            PV_69 = 0
            PV_400 = 1
            PV_868 = 0
            # EV
            num_EV = 2
            num_parkingloc = 2
            mpg_mu = 0.2
            # Energy Efficiency
            is_efficient = True
            efficiency_percent = 0.3 # Reduce values with 30%
            efficient_loads = 20
            # Flex
            is_flex = True
            flex_days = 17
            percent_flex_loads = 0.30
            flex_reduction = 0.50
            flex_only_noDH = True
        else:
            region = 'Stockholm'
            # Loads
            office = offices.value
            load_dict = {'HouseNew': housenew.value,
                         'HouseOld': houseold.value,
                         'HouseDH': housemix.value, 
                         'ApartmentNewDH': apart.value,
                        }

            # PV
            PV_69 = PV69.value
            PV_400 = PV400.value
            PV_868 = PV868.value

            # EV
            num_EV = EVnumber.value
            num_parkingloc = EVparking.value

            # EV
            num_EV = EVnumber.value
            num_parkingloc = EVparking.value
            mpg_mu = EVkwhkm.value

            # Energy Efficiency
            is_efficient = EEcheck.value
            efficiency_percent = EEreduction.value/100
            efficient_loads = EEloads.value

            # Flex
            is_flex = flexcheck.value
            flex_days = flexdays.value
            percent_flex_loads = flexloadpercent.value/100
            flex_reduction = flexpercent.value/100
            flex_only_noDH = True

        substation = scenario_substation(region,
                               load_dict, 
                               office, 
                               PV_69, 
                               PV_400, 
                               PV_868, 
                               num_EV,
                               num_parkingloc,
                               mpg_mu,
                               is_efficient, 
                               efficiency_percent, 
                               efficient_loads, 
                               is_flex, 
                               flex_days, 
                               percent_flex_loads, 
                               flex_reduction,
                               flex_only_noDH)
        
        
        nameinfobox = widgets.Box([widgets.HTML(value='<h3> Results of a simulation with 1 substation. </h3>')],
                     layout = {'margin':'15px'})

        consumptioninfobox = widgets.VBox([widgets.HTML(value='Average consumption [kWh/h]: {} (+/- {})'.format(substation.mu, substation.sigma)),
                                       widgets.HTML(value='Max consumption [kWh]: {}'.format(substation.dataframe['AggregatedLoad'].max())), 
                                       widgets.HTML(value='Total consumption [kWh]: {}'.format(substation.dataframe['AggregatedLoad'].sum())),
                                       widgets.HTML(value='Based on data from {} to {}.'.format(substation.start,substation.end)),
                                       widgets.HTML(value='The peak consumption occurs at: {}'.format(substation.find_max())),
                                      ],
                                     layout = {'margin':'15px'})

        countinfobox = widgets.VBox([widgets.HTML(value='Number of houses: {}'.format(substation.house_count)),
                                 widgets.HTML(value='Number of apartments: {}'.format(substation.apartment_count)),
                                 widgets.HTML(value='Number of offices: {}'.format(substation.office_count)),
                                 widgets.HTML(value='Number of EVs: {}'.format(substation.EV_count)),
                                 widgets.HTML(value='Number of EV charging locations: {}'.format(num_parkingloc)),],
                               layout = {'margin':'15px'})
        
        pvinfobox = widgets.Box([widgets.HTML(value='Number of PVs: {}'.format(substation.pv_count))])
        
        eeinfobox = widgets.Box([])
        flexinfobox = widgets.Box([])

        trendinfobox = widgets.VBox([pvinfobox, 
                                     eeinfobox, 
                                     flexinfobox],
                               layout = {'margin':'15px'})

        numinfobox = widgets.VBox([nameinfobox, 
                                   consumptioninfobox, 
                                   countinfobox, 
                                   trendinfobox
                                  ])
        graphinfobox = widgets.VBox([widgets.HTML('Soon to be pretty!!!')])

        tabout = widgets.Tab([numinfobox,graphinfobox])
        tabout.set_title(0, 'Info')
        tabout.set_title(1, 'Plots and Graphs')
        display(tabout)


    runbtn.on_click(lambda e: run_model(e,scenario = scenario_selection.value))
    runbtn2.on_click(lambda e: run_model(e))

    def flex_disable(event):
        if flexpercent.disabled:
            flexpercent.disabled = False
            flexdays.disabled = False
            flexloadpercent.disabled = False

        else:
            flexpercent.disabled = True
            flexdays.disabled = True
            flexloadpercent.disabled = True

    flexcheck.observe(lambda e: flex_disable(e))

    def ee_disable(event):
        if EEloads.disabled:
            EEloads.disabled = False
            EEreduction.disabled = False
        else:
            EEloads.disabled = True
            EEreduction.disabled = True 

    EEcheck.observe(lambda e: ee_disable(e))

    def load_change(event):
        change = event['new'] - event['old']
        if EEloads.max + change > 0:
            EEloads.max += change

    housenew.observe(lambda e: load_change(e), names='value')
    houseold.observe(lambda e: load_change(e), names='value')
    housemix.observe(lambda e: load_change(e), names='value')
    apart.observe(lambda e: load_change(e), names='value')
    offices.observe(lambda e: load_change(e), names='value')




    #----------------- Tabs and display --------------------------------------

    tab = widgets.Tab([scenariobox,create_scenario])
    tab.set_title(0, 'Scenario 1 - 4')
    tab.set_title(1, 'Custom Scenario')

    display(Markdown('# Model Simulation'), Markdown('Here you run the simulation. Either choose a scenario or build your own.'))
    display(tab)

SyntaxError: invalid syntax (<ipython-input-17-2b6ff28eae86>, line 333)

In [13]:
import pandas as pd
import numpy as np

from substation import Substation
from net import Net

def scenario_substation(region,
                        load_dict, 
                        offices, 
                        PV_69, 
                        PV_400, 
                        PV_868, 
                        num_EV, 
                        num_parkingloc,
                        mpg_mu,
                        is_efficient, 
                        efficiency_percent, 
                        efficient_loads, 
                        is_flex, 
                        flex_days, 
                        percent_flex_loads, 
                        flex_reduction,
                        flex_only_noDH):
    
    # Create substation
    station = Substation(region)
    
    # Add residential loads to susbstation
    for load,num in load_dict.items():
        station.add_residential_load(load,num)
        
    # Add offices
    if offices > 0:
        station.add_office(num = offices)
    
    # Add PVs
    if PV_69 > 0:
        station.add_PV(size = 69, 
                       num = PV_69)
        
    if PV_400 > 0:
        station.add_PV(size = 400, 
                       num = PV_400)
        
    if PV_868 > 0:
        station.add_PV(size = 868, 
                       num = PV_868)
    
    # Add EVs
    if num_EV > 0:
        station.add_EV(num_EV, num_parkingloc, mpg_mu)
    
    # Introduce EE
    if is_efficient:
        station.introduce_efficiency(percent = efficiency_percent, 
                                     num = efficient_loads)
    
    # Introduce Flex
    if is_flex:
        station.introduce_flexibility(days = flex_days, 
                                      percent_loads = percent_flex_loads, 
                                      reduction = flex_reduction, 
                                      only_noDH = flex_only_noDH)
    station.calculate_norm()
        
    return station